In [1]:
import numpy as np
import cv2 as cv

img = cv.imread('data/00350405_2611802704.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
sift = cv.SIFT_create()
kp = sift.detect(gray, None)
img = cv.drawKeypoints(gray, kp, img)


In [6]:
from pathlib import Path
import cv2

def read_image(path: Path, grayscale=False):
    """This function read an image from a path.
    The read is perform using opencv.
    """
    if grayscale:
        mode = cv2.IMREAD_GRAYSCALE
    else:
        mode = cv2.IMREAD_COLOR
    image = cv2.imread(str(path), mode)
    if image is None:
        raise ValueError(f'Cannot read image {path}.')
    if not grayscale and len(image.shape) == 3:
        image = image[:, :, ::-1]  # BGR to RGB
    return image

In [61]:
from types import SimpleNamespace
from pathlib import Path

class ImageSIFTDataset():
    default_conf = {
      'globs': ['*.jpg', '*.png', '*.jpeg', '*.JPG', '*.PNG'],
      'grayscale': False,
      'interpolation': 'cv2_area',
      'resize': (1024,1024)
    }

    def __init__(self, root, conf=default_conf):
        self.conf = conf = SimpleNamespace(**{**self.default_conf, **conf})
        self.root = root
        paths = []
        for g in self.conf.globs:
            paths += list(Path(root).glob('**/'+g))
        if len(paths) == 0:
            raise ValueError(f'Could not find any image in {root}.')
        paths = sorted(list(set(paths)))
        self.names = [i.relative_to(root).as_posix() for i in paths]
        self.sift = cv2.SIFT_create()
    
    def __getitem__(self, idx):
        i = self._read_image(self.root/Path(self.names[idx]))
        feat = self._compute_sift(i)
        return {
            'image' : i,
            'descriptor': feat
        }
    
    def __len__(self):
        return len(self.names)
    
    def _compute_sift(self, i):
        i = cv2.resize(i, self.conf.resize, interpolation=self._get_interpolation())
        kp, des = self.sift.detectAndCompute(i, None)
        return des
    
    def _read_image(self, path: Path):
        """This function read an image from a path.
        The read is perform using opencv.
        """
        if self.conf.grayscale:
            mode = cv2.IMREAD_GRAYSCALE
        else:
            mode = cv2.IMREAD_COLOR_RGB
        image = cv2.imread(str(path), mode)
        if image is None:
            raise ValueError(f'Cannot read image {path}.')
        return image
    
    def _get_interpolation(self):
        """Return the correct interpolation method based on the config."""
        if self.conf.interpolation == 'cv2_area':
            return cv2.INTER_AREA
        elif self.conf.interpolation == 'cv2_linear':
            return cv2.INTER_LINEAR
        elif self.conf.interpolation == 'cv2_cubic':
            return cv2.INTER_CUBIC
        else:
            raise ValueError(f"Unknown interpolation method: {self.conf.interpolation}")

In [ ]:
class vlad:
    def __init__(self, n_vocabs = 64, d = 128):
        self.n_vocabs = n_vocabs
        self.d = d
        self.n_vocabs = None
        self.centers = None

# Testing FAISS API

In [68]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [69]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [74]:
faiss.Clustering.

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__set_name__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'deleter',
 'fdel',
 'fget',
 'fset',
 'getter',
 'setter']

# Testing OpenCV API

In [28]:
sift = cv.SIFT_create()

test_path = 'data/01065157_3875793450.jpg'

test_img_bgr = cv.imread(test_path, cv.IMREAD_COLOR_BGR)
test_img_gray = cv.imread(test_path, cv.IMREAD_GRAYSCALE)
test_img_rgb = cv.imread(test_path, cv.IMREAD_COLOR_RGB)

_, test_sift_bgr = sift.detectAndCompute(test_img_bgr, None)
_, test_sift_gray = sift.detectAndCompute(test_img_gray, None)
_, test_sift_rgb = sift.detectAndCompute(test_img_rgb, None)

In [64]:
test_sift_bgr[0:20]

array([[ 0.,  0.,  7., ...,  2., 14., 32.],
       [ 4., 28., 71., ...,  2., 13., 30.],
       [ 4.,  1.,  0., ...,  1.,  6., 34.],
       ...,
       [ 4., 25., 40., ...,  0.,  0., 12.],
       [33.,  3.,  4., ...,  0.,  0.,  0.],
       [22., 12., 17., ...,  2.,  4., 11.]],
      shape=(20, 128), dtype=float32)

In [65]:
test_sift_gray[0:20]

array([[ 0.,  0.,  7., ...,  2., 14., 32.],
       [ 4., 28., 71., ...,  2., 13., 30.],
       [ 4.,  1.,  0., ...,  1.,  6., 34.],
       ...,
       [ 4., 25., 40., ...,  0.,  0., 12.],
       [33.,  3.,  4., ...,  0.,  0.,  0.],
       [22., 12., 17., ...,  2.,  4., 11.]],
      shape=(20, 128), dtype=float32)

In [67]:
test_sift_rgb[0:20]

array([[ 0.,  0.,  8., ...,  2., 14., 30.],
       [ 2.,  0.,  0., ...,  0.,  4., 27.],
       [ 0.,  1.,  7., ...,  0.,  0.,  1.],
       ...,
       [ 5., 23., 40., ...,  0.,  0., 11.],
       [34.,  3.,  4., ...,  0.,  0.,  0.],
       [17., 10., 20., ...,  2.,  3., 16.]],
      shape=(20, 128), dtype=float32)

The SIFT features generated by opencv's API doesnt differ much no matter which type of image use, just make sure your file are all the same type of image